In [2]:
import pandas as pd
import os
import numpy as np
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import requests
import time

In [3]:
# get parent directory
parent_dir = os.path.dirname(os.getcwd())

In [4]:
# import data from all csv files in the raw_data folder into dataframes
df = pd.DataFrame()
parent_dir = os.path.dirname(os.getcwd())
lst = os.listdir(parent_dir + '/raw_data')
lst.sort()
for filename in lst:
    f = os.path.join(parent_dir + '/raw_data', filename)
    if f.endswith('.csv'):
        df = pd.concat([df, pd.read_csv(f)], ignore_index=True)

In [5]:
# create a new column for the decade of each song

decades = ['1960', '1970', '1980', '1990', '2000', '2010']
decade_length = []
decade_column = []
for filename in lst:
    f = os.path.join(parent_dir + '/raw_data', filename)
    if f.endswith('.csv'):
        decade_length.append(pd.read_csv(f).shape[0])
for i in range(len(decade_length)):
    decade_column.extend([decades[i]] * decade_length[i])
df['decade'] = decade_column
print(decade_length)

# insert empty column called release_date
df.insert(3, 'release_date', None)

[8642, 7766, 6908, 5520, 5872, 6398]


In [6]:
# log into spotify api using client credentials, and get metadata for a song using its uri
client_credentials_manager = SpotifyClientCredentials(client_id='00192ef409c4480c8ecad758c9ed8478', client_secret='f6a4d11d5187418a99b8045ec84fb3b6')
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [7]:
# use requests to log into spotify api using client credentials, and get release date for a song using its uri
def get_song_info(uri):
    url = 'https://api.spotify.com/v1/tracks/' + uri.split(':')[2]
    headers = {'Authorization': 'Bearer ' + client_credentials_manager.get_access_token(as_dict=False)}
    r = requests.get(url, headers=headers)
    print(r)
    r = r.json()
    return r['album']['release_date']


In [9]:
get_song_info('spotify:track:1bx6spmieE655BQvWdTYKA')

<Response [200]>


'0000'

In [8]:
# for every row of df, add a column with the release date of the song

i = 0
while (i <= len(df)):
    try:
        df['release_date'][i] = get_song_info(df['uri'][i])
        print(i, '/', len(df), '\n')
        # write to csv every 1 row
        df.to_csv(parent_dir + '/parsed_data/songs.csv', index=False)                         
        time.sleep(0.3)
        i += 1
    except:
        time.sleep(5)

<Response [200]>
0 / 41106 



<ipython-input-8-ac175d058a91>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['release_date'][i] = get_song_info(df['uri'][i])


<Response [200]>
1 / 41106 



In [9]:
# save dataframe as csv file
df.to_csv(parent_dir + '/parsed_data/songs.csv', index=False)